In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+rocm5.6
True


In [2]:
! git clone https://github.com/aurorabuilder/elements

fatal: destination path 'elements' already exists and is not an empty directory.


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.llms import LlamaCpp
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [4]:
model = LlamaCpp(
    model_path="models/neural-chat-7b-v3-3.Q5_K_M.gguf",
    temperature=0.1,
    max_tokens=512,
    top_p=0.95,
    # callback_manager=callback_manager,
    verbose=True, # Verbose is required to pass to the callback manager
    n_gpu_layers=-1,
    echo=True,
    n_ctx=1024*32,
    stop=["### System", "### User", "### Assistant", "User:", "Player:", "DM:"],
)

ggml_opencl: selecting platform: 'AMD Accelerated Parallel Processing'
ggml_opencl: selecting device: 'gfx1030'
ggml_opencl: device FP16 support: true
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/neural-chat-7b-v3-3.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = intel_neural-chat-7b-v3-3
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336

In [20]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [21]:
from langchain.document_loaders import PyPDFLoader, UnstructuredXMLLoader

import os 
import glob  
# Define the directory path 
directory_path = 'elements/core/players-handbook'  
# Initialize a list to store the XML file contents as strings 
xml_strings = []  
# Use the glob module to recursively find XML files in the directory and its subfolders 
xml_files = glob.glob('*.xml', root_dir=directory_path, recursive=True)

documents = []

for xml_file in xml_files:
    loader = UnstructuredXMLLoader(os.path.join(directory_path, xml_file))
    documents.extend(loader.load())
# Loop through each XML file and read its content as a string 
# for xml_file in xml_files:     
#     with open(os.path.join(directory_path, xml_file), 'r', encoding='utf-8') as file:         
#         xml_string = file.read()         
#         xml_strings.append(xml_string)  
# Now, the xml_strings list contains the contents of all XML files as strings

In [22]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)
docs[0]

Created a chunk of size 2192, which is longer than the specified 1024
Created a chunk of size 1089, which is longer than the specified 1024
Created a chunk of size 1220, which is longer than the specified 1024
Created a chunk of size 1150, which is longer than the specified 1024
Created a chunk of size 1253, which is longer than the specified 1024
Created a chunk of size 1259, which is longer than the specified 1024
Created a chunk of size 1256, which is longer than the specified 1024
Created a chunk of size 1362, which is longer than the specified 1024
Created a chunk of size 1217, which is longer than the specified 1024
Created a chunk of size 1233, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024
Created a chunk of size 2253, which is longer than the specified 1024
Created a chunk of size 1428, which is longer than the specified 1024
Created a chunk of size 1245, which is longer than the specified 1024
Created a chunk of s

Document(page_content='Companions\nCompanion creatures from the Player’s Handbook.\nWizards of the Coast\n16\n14\n14\n6\n12\n7\n12\n19 (3d8+6)\n30 ft., climb 30 ft.\n—\nAthletics +5, Perception +3\nBeast\nMedium\nunaligned\n1/2\nID_WOTC_PHB_COMPANION_ACTION_APE_MULTIATTACK,ID_WOTC_PHB_COMPANION_ACTION_APE_FIST,ID_WOTC_PHB_COMPANION_ACTION_APE_ROCK\nThe ape makes two fist attacks.\nThe ape makes two fist attacks.\nMelee Weapon Attack: +5 to hit, reach 5 ft., one target. Hit: 6 (1d6 + 3) bludgeoning damage.\nMelee Weapon Attack: +5 to hit, reach 5 ft., one target. Hit: 6 (1d6 + 3) bludgeoning damage.\nRanged Weapon Attack: +5 to hit, range 25/50 ft., one target. Hit: 6 (1d6 + 3) bludgeoning damage.\nRanged Weapon Attack: +5 to hit, range 25/50 ft., one target. Hit: 6 (1d6 + 3) bludgeoning damage.\n8\n14\n11\n4\n12\n6\n12\n3 (1d6)\n30 ft., climb 30 ft.\n—\nBeast\nSmall\nunaligned\n0\nID_WOTC_PHB_COMPANION_TRAIT_BABOON_PACK_TACTICS\nID_WOTC_PHB_COMPANION_ACTION_BABOON_BITE', metadata={'sou

In [27]:
document_texts = [doc.page_content for doc in docs]
vectorstore = FAISS.from_texts(texts=document_texts, embedding=hf)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# TODO cohere is a paid API. Need to figure out how to replace it with an OS rerank model like https://huggingface.co/BAAI/bge-reranker-large
# TODO probably we can extend the BaseDocumentCompressor to use a reranker HF model
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ValidationError: 1 validation error for CohereRerank
__root__
  Did not find cohere_api_key, please add an environment variable `COHERE_API_KEY` which contains it, or pass `cohere_api_key` as a named parameter. (type=value_error)

In [25]:
template = """### System:
Answer the user's question based only on the following context:
{context}

### User: Question: {question}

### Assistant:
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
from langchain.globals import set_verbose, set_debug
set_debug(True)
chain.invoke("What is the attack bonus of a LVL 2 Barbarian?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the attack bonus of a LVL 2 Barbarian?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What is the attack bonus of a LVL 2 Barbarian?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What is the attack bonus of a LVL 2 Barbarian?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What is the attack bonus of a LVL 2 Barbarian?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chai

Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 6:llm:LlamaCpp] [53.51s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The given context does not specifically mention any details about a Level 2 Barbarian's attack bonus.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 7:parser:StrOutputParser] Entering Parser run with input:
{
  "input": " The given context does not specifically mention any details about a Level 2 Barbarian's attack bonus."
}
[chain/end] [1:chain:RunnableSequence > 7:parser:StrOutputParser] s] Exiting Parser run with output:
{
  "output": " The given context does not specifically mention any details about a Level 2 Barbarian's attack bonus."
}
[chain/end] [1:chain:RunnableSequence] [53.52s] Exiting Chain run with output:
{
  "output": " The given context does not specifically mention any details about a Level 2 Barbarian's attack 


llama_print_timings:        load time =     971.57 ms
llama_print_timings:      sample time =       2.87 ms /    23 runs   (    0.12 ms per token,  8027.92 tokens per second)
llama_print_timings: prompt eval time =   52303.31 ms /   816 tokens (   64.10 ms per token,    15.60 tokens per second)
llama_print_timings:        eval time =    1038.48 ms /    22 runs   (   47.20 ms per token,    21.18 tokens per second)
llama_print_timings:       total time =   53505.86 ms


" The given context does not specifically mention any details about a Level 2 Barbarian's attack bonus."

In [7]:
import torch
torch.__version__

'2.1.2+rocm5.6'

In [8]:
!python -c "import torch;print(torch.cuda.is_available());"

True


In [9]:
torch.cuda.is_available()

False